## Introduction to Massive Data Analysis 
### HW1 - Matrix Multiplication
106070038 杜葳葳

匯入 SparkConf、SparkContext、Pyspark模組 <br>
設定環境變數（解決版本問題）

In [1]:
# import package
from pyspark import SparkConf,SparkContext
import pyspark

# set environment variables
import os
os.environ["PYSPARK_PYTHON"]="/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8"
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8'

確定檔案存在

In [2]:
# check whether .txt file exist
## d = open('500input.txt', mode='r')
## words = d.read(18)
## print(words)

In [3]:
sc.stop()

**建立 SparkConf 物件** <br>
設為 local 模式、名稱設為 "hw1"

In [4]:
conf = SparkConf().setMaster("local").setAppName("hw1")
sc = SparkContext(conf=conf)
sc

<SparkContext master=local appName=hw1>

### 讀檔

In [5]:
lines = sc.textFile("500input.txt")

### Transformation
用 .split(",") 把資料切開 <br>
ex. M,0,0,51 -> ['M','0','0','51'] <br>
> **map (function)** <br>
> Return a new distributed dataset formed by passing each element of the source through a function func.

In [6]:
Split_RDD = lines.map(lambda line : line.split(","))

In [8]:
# Split_RDD.collect()

[['M', '0', '0', '51'],
 ['M', '0', '1', '17'],
 ['M', '0', '2', '83'],
 ['M', '0', '3', '97'],
 ['M', '0', '4', '20'],
 ['M', '0', '5', '52'],
 ['M', '0', '6', '85'],
 ['M', '0', '7', '65'],
 ['M', '0', '8', '42'],
 ['M', '0', '9', '8'],
 ['M', '0', '10', '18'],
 ['M', '0', '11', '53'],
 ['M', '0', '12', '69'],
 ['M', '0', '13', '85'],
 ['M', '0', '14', '89'],
 ['M', '0', '15', '51'],
 ['M', '0', '16', '81'],
 ['M', '0', '17', '25'],
 ['M', '0', '18', '43'],
 ['M', '0', '19', '38'],
 ['M', '0', '20', '43'],
 ['M', '0', '21', '2'],
 ['M', '0', '22', '91'],
 ['M', '0', '23', '56'],
 ['M', '0', '24', '15'],
 ['M', '0', '25', '77'],
 ['M', '0', '26', '19'],
 ['M', '0', '27', '91'],
 ['M', '0', '28', '23'],
 ['M', '0', '29', '81'],
 ['M', '0', '30', '49'],
 ['M', '0', '31', '10'],
 ['M', '0', '32', '73'],
 ['M', '0', '33', '55'],
 ['M', '0', '34', '56'],
 ['M', '0', '35', '12'],
 ['M', '0', '36', '73'],
 ['M', '0', '37', '60'],
 ['M', '0', '38', '12'],
 ['M', '0', '39', '76'],
 ['M', '0', 

用 key（'M','N'）來區分兩個矩陣，再用兩個新的 RDD 存
> **.filter(function)** <br>
> Return a new dataset formed by selecting those elements of the source on which func returns true.

In [7]:
M_RDD = Split_RDD.filter(lambda x : "M" in x[0])
N_RDD = Split_RDD.filter(lambda x : "N" in x[0])

### Matrix Multiplication
M和N兩個矩陣相乘,得到乘積P=MN<br>
M矩陣有i個row,j個col<br>
N矩陣有j個row,k個col<br>
則P矩陣的每個element(p)<br>
#### p[i][k]=∑m[i][j]*n[j][k]<br>

觀察上式,發現當**M矩陣的col和N矩陣的row相同時（都為j）** 要相乘,故將兩個矩陣的key都改為j,方便後面做運算

### Map
x[0]:'M', x[1]:row(i), x[2]:col(j), x[3]:value_ij <br>
整理為 (j, ['M',i,value_ij] ) <br>
key=j, value = ['M',i,value_ij]

In [8]:
M_RDD_new = M_RDD.map(lambda x : (x[2],[ x[0],x[1],x[3]] ))

In [10]:
# M_RDD_new.collect()

[('0', ['M', '0', '51']),
 ('1', ['M', '0', '17']),
 ('2', ['M', '0', '83']),
 ('3', ['M', '0', '97']),
 ('4', ['M', '0', '20']),
 ('5', ['M', '0', '52']),
 ('6', ['M', '0', '85']),
 ('7', ['M', '0', '65']),
 ('8', ['M', '0', '42']),
 ('9', ['M', '0', '8']),
 ('10', ['M', '0', '18']),
 ('11', ['M', '0', '53']),
 ('12', ['M', '0', '69']),
 ('13', ['M', '0', '85']),
 ('14', ['M', '0', '89']),
 ('15', ['M', '0', '51']),
 ('16', ['M', '0', '81']),
 ('17', ['M', '0', '25']),
 ('18', ['M', '0', '43']),
 ('19', ['M', '0', '38']),
 ('20', ['M', '0', '43']),
 ('21', ['M', '0', '2']),
 ('22', ['M', '0', '91']),
 ('23', ['M', '0', '56']),
 ('24', ['M', '0', '15']),
 ('25', ['M', '0', '77']),
 ('26', ['M', '0', '19']),
 ('27', ['M', '0', '91']),
 ('28', ['M', '0', '23']),
 ('29', ['M', '0', '81']),
 ('30', ['M', '0', '49']),
 ('31', ['M', '0', '10']),
 ('32', ['M', '0', '73']),
 ('33', ['M', '0', '55']),
 ('34', ['M', '0', '56']),
 ('35', ['M', '0', '12']),
 ('36', ['M', '0', '73']),
 ('37', ['M',

### Map
x[0]:'N', x[1]:row(j), x[2]:col(k), x[3]:value_jk <br>
整理為 (j, ['N',k,value_jk] ) <br>
key=j, value = ['N',k,value_jk]

In [9]:
N_RDD_new = N_RDD.map(lambda x : (x[1], [x[0],x[2],x[3]] ))

In [12]:
# N_RDD_new.collect()

[('0', ['N', '0', '73']),
 ('0', ['N', '1', '58']),
 ('0', ['N', '2', '23']),
 ('0', ['N', '3', '5']),
 ('0', ['N', '4', '65']),
 ('0', ['N', '5', '95']),
 ('0', ['N', '6', '70']),
 ('0', ['N', '7', '55']),
 ('0', ['N', '8', '59']),
 ('0', ['N', '9', '28']),
 ('0', ['N', '10', '0']),
 ('0', ['N', '11', '55']),
 ('0', ['N', '12', '87']),
 ('0', ['N', '13', '41']),
 ('0', ['N', '14', '84']),
 ('0', ['N', '15', '99']),
 ('0', ['N', '16', '0']),
 ('0', ['N', '17', '30']),
 ('0', ['N', '18', '27']),
 ('0', ['N', '19', '46']),
 ('0', ['N', '20', '88']),
 ('0', ['N', '21', '25']),
 ('0', ['N', '22', '83']),
 ('0', ['N', '23', '85']),
 ('0', ['N', '24', '94']),
 ('0', ['N', '25', '32']),
 ('0', ['N', '26', '86']),
 ('0', ['N', '27', '64']),
 ('0', ['N', '28', '40']),
 ('0', ['N', '29', '47']),
 ('0', ['N', '30', '45']),
 ('0', ['N', '31', '68']),
 ('0', ['N', '32', '76']),
 ('0', ['N', '33', '43']),
 ('0', ['N', '34', '75']),
 ('0', ['N', '35', '16']),
 ('0', ['N', '36', '76']),
 ('0', ['N', '

### Reduce
M_RDD_new: key=j, value=['M',i,value_ij] <br>
N_RDD_new: key=j, value=['N',k,value_jk] <br>
把兩個RDD join在一起,得到MN_RDD 
#### key = j, value = (['M', i, value_ij], ['N', k, value_jk])

> **join(otherDataset, [numPartitions])** <br>
> When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key.
> >Perform an inner join between two RDDs.<br>
> >join是先union兩個rdd,再groupByKey()

In [10]:
MN_RDD = M_RDD_new.join(N_RDD_new)

接著把每個pair中的value_ij和value_jk相乘得到ANS_RDD_matrix <br>
key =（i,k）, value = value_ij * value_jk <br>

In [11]:
ANS_RDD_matrix = MN_RDD.map(lambda x: ((int(x[1][0][1]),int(x[1][1][1])),int(x[1][0][2])*int(x[1][1][2])) )

### Reduce
把所有key相同的value加總
> **reduceByKey(func, [numPartitions])** <br>
> When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V,V) => V.

In [12]:
ANS_RDD = ANS_RDD_matrix.reduceByKey(lambda x,y: int(x)+int(y))

In [17]:
# ANS_RDD.collect()

[((0, 0), 1224217),
 ((0, 2), 1172528),
 ((0, 4), 1152989),
 ((0, 6), 1192906),
 ((0, 8), 1146928),
 ((0, 10), 1164367),
 ((0, 12), 1197989),
 ((0, 14), 1169192),
 ((0, 16), 1170027),
 ((0, 18), 1246305),
 ((0, 20), 1193495),
 ((0, 22), 1188082),
 ((0, 24), 1171027),
 ((0, 26), 1172161),
 ((0, 28), 1158241),
 ((0, 30), 1207314),
 ((0, 32), 1212365),
 ((0, 34), 1260592),
 ((0, 36), 1177580),
 ((0, 38), 1128280),
 ((0, 40), 1174699),
 ((0, 42), 1208107),
 ((0, 44), 1152943),
 ((0, 46), 1154121),
 ((0, 48), 1231126),
 ((0, 50), 1158492),
 ((0, 52), 1143906),
 ((0, 54), 1194344),
 ((0, 56), 1174863),
 ((0, 58), 1181720),
 ((0, 60), 1238751),
 ((0, 62), 1152540),
 ((0, 64), 1189808),
 ((0, 66), 1138804),
 ((0, 68), 1222778),
 ((0, 70), 1178505),
 ((0, 72), 1183000),
 ((0, 74), 1213413),
 ((0, 76), 1115773),
 ((0, 78), 1222045),
 ((0, 80), 1180707),
 ((0, 82), 1190793),
 ((0, 84), 1187149),
 ((0, 86), 1181296),
 ((0, 88), 1195005),
 ((0, 90), 1202149),
 ((0, 92), 1149511),
 ((0, 94), 1189661

把結果依照ascending排序
> **sortByKey([ascending], [numPartitions])** <br>
> When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in the boolean ascending argument.

In [13]:
P_RDD_sort = ANS_RDD.sortByKey()

In [14]:
P_RDD_sort.collect()

[(('0', '0'), 1224217),
 (('0', '1'), 1209205),
 (('0', '10'), 1164367),
 (('0', '100'), 1178902),
 (('0', '101'), 1222956),
 (('0', '102'), 1237878),
 (('0', '103'), 1178114),
 (('0', '104'), 1150404),
 (('0', '105'), 1193564),
 (('0', '106'), 1185107),
 (('0', '107'), 1160686),
 (('0', '108'), 1173207),
 (('0', '109'), 1193923),
 (('0', '11'), 1197043),
 (('0', '110'), 1209783),
 (('0', '111'), 1203917),
 (('0', '112'), 1194381),
 (('0', '113'), 1169108),
 (('0', '114'), 1269440),
 (('0', '115'), 1084155),
 (('0', '116'), 1125298),
 (('0', '117'), 1137173),
 (('0', '118'), 1214441),
 (('0', '119'), 1166521),
 (('0', '12'), 1197989),
 (('0', '120'), 1214331),
 (('0', '121'), 1159342),
 (('0', '122'), 1176343),
 (('0', '123'), 1141896),
 (('0', '124'), 1245100),
 (('0', '125'), 1137977),
 (('0', '126'), 1149950),
 (('0', '127'), 1161472),
 (('0', '128'), 1221775),
 (('0', '129'), 1194020),
 (('0', '13'), 1132939),
 (('0', '130'), 1208966),
 (('0', '131'), 1164026),
 (('0', '132'), 1142

將RDD映射至同層

In [14]:
P_RDD_reshape = P_RDD_sort.map(lambda x : (x[0][0],x[0][1],x[1]) )

In [15]:
output = P_RDD_reshape.collect()

### Output
RDD形成2個分區,saveAsTextFile()會產生part-0000、part-0001以及_SUCCESS這三個文件 <br>
part-0000、part-0001包含了RDD中所有的資料 <br>
_SUCCESS是一個空文件,用來表示寫入操作執行成功 <br>

In [16]:
P_RDD_reshape.saveAsTextFile("Output")

In [17]:
sc.stop()

### Note
Spark的函式主要分成**Transformation**和**Action**兩類 <br>
Transformation: 資料轉換類函式 <br>
Action: 行動類函式(collect) <br>
**全部步驟做完,最後再一次collect()